This notebook takes an array of coldkeys and finds the emission on every hotkey associated with the coldkey.
This works for miners/validators and also stakeholders.

1. ADd your API key in the next cell (get one at dash.taostats.io)
2. choose your start and end dates in the format YYYY-MM-DD
3. Create an array of coldkeys that you wish to track

What happens:

first cell - stup the urls
2nd cell - silly date fixer - sometimes the chain adds MS, someties it does not :man shrugging:
3rd cell - query the API to get the daily close price for tao for the date range
4th: initialise the data
5th: Get the dailye data for each coldkey. There is a slepp to prevent any API rate limiting (after a few price calls that might happen)
6th: print one result
7th: math the daiy change by comparing day over day balance changes.
8th: get any registrations and caluclate those
9 & 10 - create a JSON and CSV for the created data

coldkey, hotkey, date, income, expense, price

your CK, the HK associated, day, tao earned, tao spent, tao price in $

the hotkey can be your miner, your validator.. if you are staking it is vthe hotkey of the validator you staked with.


In [ ]:
import requests, json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import decimal
import time
from datetime import datetime, timedelta
from rich import print
api_key=""
start_date= "2023-01-01"
end_date="2024-11-15"

coldkeys =[

"coldkey1", "coldkey2"
    
]

url = f"https://api.taostats.io/api/accounting/v1?date_start={start_date}&date_end={end_date}&coldkey={coldkey}"

headers = {
            "accept": "application/json",
            "Authorization": api_key
        }


In [ ]:
#sometimes the chain spits out a time with milliseconds.. Sometimess it doesnt.  Its annoying.
#this fixes it. returns a datetime object
def stupid_time_fix(timestamp):
    if len(timestamp) > 20:
        date_obj = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
    else:
        date_obj = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ")
    
    return date_obj

In [ ]:
#get the close price of tao for every date in the range

date_obj = datetime.strptime(start_date, "%Y-%m-%d")
unix_start = int(date_obj.timestamp())
date_obj = datetime.strptime(end_date, "%Y-%m-%d")
unix_end = int(date_obj.timestamp())

print(unix_start, unix_end)

#get all the tao prices
all_price_data = []

next_page=1

while next_page is not None:
    price_url=f"https://api.taostats.io/api/price/ohlc/v1?asset=tao&period=1d&timestamp_start={unix_start}&timestamp_end={unix_end}&limit=200&page={next_page}"
    response = requests.get(price_url, headers=headers)
    #print(response.text)
    resJson = json.loads(response.text)
    next_page = resJson['pagination']['next_page']
    all_price_data += resJson['data']

print(len(all_price_data))
date_price = {}
for price in all_price_data:
    pricetime  = stupid_time_fix(price['timestamp'])
    date = pricetime.date()
    date_str = date.strftime("%Y-%m-%d")
    close = price['close']
    date_price[date_str]=close

In [ ]:
#print(resJson)
income_totals = {}
daily_stake = {}
stake_balances = []
neuron_registrations = []

In [ ]:

for coldkey in coldkeys:
    pages = 1
    page =1

    url = f"https://api.taostats.io/api/accounting/v1?date_start={start_date}&date_end={end_date}&coldkey={coldkey}&page={page}"
    
    response = requests.get(url, headers=headers)
    #print(response.text)
    resJson = json.loads(response.text)
    income = float(resJson['data'][0]['income'])/1e9
    reg_costs = float(resJson['data'][0]['neuron_registration_cost'])/1e9
    pages = resJson['pagination']['total_pages']
    print(coldkey, income, reg_costs)
    stake_balances += resJson['data'][0]['stake_balances']
    neuron_registrations  += resJson['data'][0]['neuron_registrations']
    time.sleep(3)

In [ ]:
print(stake_balances[0])

In [ ]:
total =0

initial_stake = float(stake_balances[0]['stake'])/1e9
for index, balance in enumerate(stake_balances):
    if index >0:
        timestamp = stupid_time_fix(balance['timestamp'])
        date = timestamp.date()
        date_str = date.strftime("%Y-%m-%d")
        hk = balance['hotkey']['ss58']
        ck= balance['coldkey']['ss58']
        if hk not in income_totals:
            income_totals[hk] = {}
        #ignore first day, we have that inital stake
        instantaneous_stake = float(balance['stake'])/1e9 - float(balance['added'])/1e9 + float(balance['removed'])/1e9
        if not hk in daily_stake:
            daily_stake[hk]=[]
        daily_stake[hk].append(float(balance['stake'])/1e9)
        # with many hotkey cannot assume last entry
        #past_instantaneous_stake =float(stake_balances[index-1]['stake'])/1e9
        #grab day before' data from the incometotals

        if len(daily_stake[hk])>1:
            past_instantaneous_stake = daily_stake[hk][-2] 
        else:
            past_instantaneous_stake =0

        interval_increase = instantaneous_stake - past_instantaneous_stake
        if hk == '5GKddwB5MjywC4HSo6T3nRBfvP641XzaCuFw7VbwHv7dDQXy':
            
            print(date_str, hk,round(past_instantaneous_stake,3), round(instantaneous_stake,3),round(interval_increase,3))
        ##interval increase is the income for that day
        if date_str in income_totals[hk] and ck in income_totals[hk]:
            if 'income' in income_totals[hk][date_str]:
                income_totals[hk][date_str]['income'] += round(interval_increase,6)
            else:
                #new entry
                income_totals[hk][date_str]['income'] = round(interval_increase,6)
                income_totals[hk][date_str]['price'] =  date_price[date_str]   
                income_totals[hk][date_str]['coldkey'] = ck
        else:
            #new data and enw expense
            income_totals[hk][date_str] = {}
            income_totals[hk][date_str]['income'] = round(interval_increase,6)
            income_totals[hk][date_str]['price'] =  date_price.get(date_str,0)
            income_totals[hk][date_str]['coldkey'] = ck

In [ ]:

for reg in neuron_registrations:
    timestamp = stupid_time_fix(reg['timestamp'])
    date = timestamp.date()
    date_str = date.strftime("%Y-%m-%d")
    reg_cost = float(reg['registration_cost'])/1e9
    hk = reg['hotkey']['ss58']
    ck = reg['coldkey']['ss58']
    if hk not in income_totals:
        income_totals[hk] = {}
    if date_str in income_totals[hk]:
        if 'expense' in income_totals[hk][date_str]:
            income_totals[hk][date_str]['expense'] += round(reg_cost,3)
        else:
            #new entry
            income_totals[hk][date_str]['expense'] = round(reg_cost,3)
            income_totals[hk][date_str]['price'] =  date_price[date_str]
            income_totals[hk][date_str]['coldkey'] = ck
    else:
        #new data and enw expense
        income_totals[hk][date_str] = {}
        income_totals[hk][date_str]['expense'] = round(reg_cost,3)
        income_totals[hk][date_str]['price'] =  date_price[date_str]
        income_totals[hk][date_str]['coldkey'] = ck


In [ ]:
# Open a text file and write the JSON object to it
with open('incometotals.json', 'w') as f:
    json.dump(income_totals, f)
    

In [ ]:
totals_csv = "coldkey , hotkey , date , income , expense , price \n"
for key, value in  income_totals.items():
    hotkey = key
    dates = value
    for key1, value1 in dates.items():
        date = key1
        prices = value1
        coldkey = prices.get('coldkey',0)
        income = prices.get('income',0)
        expense = prices.get('expense',0)
        price = prices.get('price',0)
        totals_csv += f"{coldkey} , {hotkey}, {date}, {income}, {expense}, {price} \n"
    
      
       
        

In [ ]:
import csv
import io

with open('incometotals.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    reader = csv.reader(io.StringIO(totals_csv), delimiter=',', quotechar='"')
    for row in reader:
        writer.writerow(row)